In [32]:
#function

#general table: input = int i
#top and buttom = in the same page

def location(i, page_no, top_bottom):
    if top_bottom == 'top':
        word=doc.get_toc()[i][1]
        page=doc[page_no]
        loc = page.search_for(word)
        lol=False
        if len(loc)==0:
            if 'ำ' in word:
                word=sorted(doc.get_toc()[i][1].split('ำ'), key=len)[-1]
                loc=page.search_for(word)
                if len(loc) != 0:
                    loc=page.search_for(word)[-1].y1
                else:
                    lol=True
            else:
                word=word.split(' ')
                if len(word)>=2:
                    word=' '.join(word[:2])
                    loc=page.search_for(word)
                    if len(loc) !=0:
                        loc=page.search_for(word)[-1].y1
                    else:
                        lol=True
                else:
                    lol=True
        else:
            loc=page.search_for(word)[-1].y1
        if lol:
            return 'error'
        else:
            return loc
    else:
        word=doc.get_toc()[i][1]
        page=doc[page_no]
        loc = page.search_for(word)
        lol=False
        if len(loc)==0:
            if 'ำ' in word:
                word=sorted(doc.get_toc()[i][1].split('ำ'), key=len)[-1]
                loc=page.search_for(word)
                if len(loc) != 0:
                    loc=page.search_for(word)[-1].y0
                else:
                    lol=True
            else:
                word=word.split(' ')
                if len(word)>=2:
                    word=' '.join(word[:2])
                    loc=page.search_for(word)
                    if len(loc) !=0:
                        loc=page.search_for(word)[-1].y0
                    else:
                        lol=True
                else:
                    lol=True
        else:
            loc=page.search_for(word)[-1].y0
        if lol:
            return 'error'
        else:
            return loc
    
    

def general(i):
    #not the last saraban
    if i != len(doc.get_toc())-1:
        out=[]
        numpage=doc.get_toc()[i][2]
        page=doc[numpage-1]
        lol=False

        if location(i, numpage-1, 'top') != 'error':
            top= location(i, numpage-1, 'top')
        else:
            lol=True

        if location(i+1, numpage-1, 'buttom') != 'error':
            buttom= location(i+1, numpage-1, 'buttom')
        else:
            lol=True


        if not lol:
            if top < buttom:
                df = read_pdf(path, encoding = 'utf-8',stream=True, pages=numpage,
                              area=(top, 0, buttom, 595.22), relative_area=False)
                for hh in df:
                    if not hh.empty:
                        out.append(hh)

        if len(out) != 0:
            return pd.concat(out)
        else:
            return out
    #last saraban + same page (last page)
    else:
        out=[]
        numpage=doc.get_toc()[i][2]
        page=doc[numpage-1]
        lol=False
        
        if location(i, numpage-1, 'top') != 'error':
            top= location(i, numpage-1, 'top')
        else:
            lol=True
            
        if not lol:
            df = read_pdf(path, encoding = 'utf-8',stream=True, pages=numpage,
                          area=(top, 0, 842, 595.22), relative_area=False)
            for hh in df:
                if not hh.empty:
                    out.append(hh)
        if len(out) !=0:
            return pd.concat(out)
        else:
            return out
        

#table, top/ buttom not the same page
def table(i, page_next, page_now):
    if page_next != page_now:
        out=[]
        lol=False

        if location(i, page_now, 'top') != 'error':
            top= location(i, page_now, 'top')
        else:
            lol=True
        if not lol :
            df = read_pdf(path, encoding = 'utf-8',stream=True, pages=page_now+1,
                          area=(top, 0, 842, 595.22), relative_area=False)


            for t in df:
                if not t.empty:
                    out.append(t)


        for h in range(page_next-page_now):
            #not the last page
            if h != page_next-page_now-1:
                df = read_pdf(path, encoding = 'utf-8',stream=True, pages=page_now+2+h,
                              area=(0, 0, 842, 595.22), relative_area=False)


                for hh in df:
                    if not hh.empty:
                        out.append(hh)

            #the last page
            else:
                #if not the last saraban
                if page_next != doc.page_count-1:

                    if location(i+1, page_next, 'buttom') != 'error':
                        buttom= location(i+1, page_next, 'buttom')
                    else:
                        lol=True

                    if not lol:
                        df = read_pdf(path, encoding = 'utf-8',stream=True, pages=page_now+2+h,
                                      area=(0, 0, buttom, 595.22), relative_area=False)


                        for q in df:
                            if not q.empty:
                                out.append(q)
                # if the last saraban
                else:
                    df = read_pdf(path, encoding = 'utf-8',stream=True, pages=page_now+2+h,
                                  area=(0, 0, 842, 595.22), relative_area=False)
                    for q in df:
                        if not q.empty:
                            out.append(q)
        if len(out) != 0:
            return pd.concat(out)
        else:
            return out
    else:
        return general(i)

    
def proc():
    page_now=doc.get_toc()[i][2]-1
    #print('page_now', page_now+1)
    
    #not the last saraban
    if i != len(doc.get_toc())-1:
        page_next = doc.get_toc()[i+1][2]-1
        #print('page_next', page_next+1)
        
        #if same page
        if page_now == page_next:
            return table(i, page_next, page_now)
            
         #if not the same page
        else:
            return table(i, page_next, page_now)
            
    #the last saraban         
    else:
        page_next = doc.page_count-1
        #print('page_next', page_next+1)
        #if same page
        if page_now  == page_next:
            return table(i, page_next, page_now)
            
        #if not same page
        else:
            return table(i, page_next, page_now)

In [33]:
pool='C:\\Users\\saeth\\Desktop\\ext3\\pdf\\unit3\\'
folders=sorted(os.listdir(pool))
#doc = fitz.open(path)



iteration=-1
for year in folders[:4]:
    folder=sorted(os.listdir(pool+year))
    files=random.choices(folder, k=3)
    for file in files:
        start = time.time()
        path=pool + year + '\\' + file
        doc=fitz.open(path)
        iteration+=1
        print('iter:', iteration)
        print(path)

        for i in range(len(doc.get_toc())):

            if doc.get_toc()[i][0]==3:
                proc()

            elif doc.get_toc()[i][0]==4:
                proc()

            elif doc.get_toc()[i][0]==5:
                proc()

        stop=time.time()
        print('time(sec): ',stop - start)

iter: 0
C:\Users\saeth\Desktop\ext3\pdf\unit3\2550\1509535188_1245.pdf
time(sec):  81.55703401565552
iter: 1
C:\Users\saeth\Desktop\ext3\pdf\unit3\2550\1509535292_9016.pdf
time(sec):  1642.90256857872
iter: 2
C:\Users\saeth\Desktop\ext3\pdf\unit3\2550\1509535619_7122.pdf
time(sec):  1242.32998752594
iter: 3
C:\Users\saeth\Desktop\ext3\pdf\unit3\2551\1509685081_8040.pdf
time(sec):  85.30602192878723
iter: 4
C:\Users\saeth\Desktop\ext3\pdf\unit3\2551\1509691009_7642.pdf
time(sec):  1318.3861949443817
iter: 5
C:\Users\saeth\Desktop\ext3\pdf\unit3\2551\1509690305_3477.pdf
time(sec):  69.29110217094421
iter: 6
C:\Users\saeth\Desktop\ext3\pdf\unit3\2552\1509531885_2442.pdf
time(sec):  1222.9052000045776
iter: 7
C:\Users\saeth\Desktop\ext3\pdf\unit3\2552\1509531135_7040.pdf
time(sec):  1617.6292514801025
iter: 8
C:\Users\saeth\Desktop\ext3\pdf\unit3\2552\1509531135_7040.pdf
time(sec):  3142.790982246399
iter: 9
C:\Users\saeth\Desktop\ext3\pdf\unit3\2553\1509529401_5247.pdf
time(sec):  3149.82

In [4]:
pool='C:\\Users\\saeth\\Desktop\\ext3\\pdf\\unit3\\'
folders=sorted(os.listdir(pool))
folders[:4]

['2550', '2551', '2552', '2553']

In [5]:
folders[4:8]

['2554', '2555', '2556', '2557']

In [7]:
folders[8:12]

['2558', '2559', '2560', '2561']

In [8]:
folders[12:]

['2562', '2563', '2564']

In [9]:
for year in folders[:4]:
    folder=sorted(os.listdir(pool+year))
    print(folder)

['1509534520_1852.pdf', '1509534561_3573.pdf', '1509534624_8822.pdf', '1509534911_1352.pdf', '1509535048_3721.pdf', '1509535088_6263.pdf', '1509535137_6801.pdf', '1509535188_1245.pdf', '1509535292_9016.pdf', '1509535404_4681.pdf', '1509535459_2356.pdf', '1509535562_7523.pdf', '1509535619_7122.pdf', '1509535664_3132.pdf', '1530599278_7216.pdf']
['1509684931_3084.pdf', '1509685081_8040.pdf', '1509685104_3081.pdf', '1509690144_9198.pdf', '1509690180_3167.pdf', '1509690305_3477.pdf', '1509690341_4297.pdf', '1509690427_8046.pdf', '1509690652_9347.pdf', '1509690693_6483.pdf', '1509690977_5919.pdf', '1509691009_7642.pdf', '1509691030_8009.pdf', '1509691088_7875.pdf', '1509691210_9330.pdf']
['1509530410_4306.pdf', '1509530701_8820.pdf', '1509530766_3440.pdf', '1509530862_4302.pdf', '1509530899_6833.pdf', '1509531037_2854.pdf', '1509531103_4970.pdf', '1509531135_7040.pdf', '1509531172_7753.pdf', '1509531430_5467.pdf', '1509531885_2442.pdf', '1509531942_1562.pdf', '1509531993_3438.pdf', '1509532

In [19]:

random.choices(folder, k=3)

['1509529896_5756.pdf', '1509529663_8648.pdf', '1509528638_4901.pdf']